Creating Sagemaker endpoint from custom model:
link: https://sagemaker-examples.readthedocs.io/en/latest/advanced_functionality/xgboost_bring_your_own_model/xgboost_bring_your_own_model.html

In [2]:
import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="SageMaker-MLops")["Role"]["Arn"]

bucket = "behaviour-project"
print(f"region:{region} \nrole:{role} \nbucket:{bucket}")

Couldn't call 'get_role' to get Role ARN from role name ashrith-iam to get Role path.


region:eu-central-1 
role:arn:aws:iam::444133344330:role/service-role/SageMaker-MLops 
bucket:behaviour-project


In [2]:
s3 = boto3.resource("s3")
try:
    if region == "eu-central-1":
        s3.create_bucket(
            Bucket=bucket,
            CreateBucketConfiguration={"LocationConstraint": "eu-central-1"},
        )
    print("S3 bucket created successfully")
except Exception as e:
    print("S3 error: ", e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [3]:
prefix = "DEMO-xgboost"
bucket_path = "https://s3.{}.amazonaws.com/{}/{}".format(region, bucket, prefix)
print(f"bucket path:{bucket_path}")
# customize to your bucket where you have stored the data

bucket path:https://s3.eu-central-1.amazonaws.com/behaviour-project/DEMO-xgboost


In [4]:
loss = lambda y, y_pred: np.sum(abs(np.subtract(np.array(y), np.array(y_pred))))
remove_first_col = (
    lambda df: df[[df.columns[i] for i in range(len(df.columns)) if i != 0]]
    .astype(str)
    .astype(int)
)

In [5]:
import pandas as pd
import numpy as np

path_to_y = "../labels/labels_y/"
path_to_x = "../labels/labels_x/"

files = sorted(os.listdir(path_to_y))
# creating data frame with columns:'frame','food', 'media', 'transaction'
y_full = pd.DataFrame(
    np.column_stack(
        [
            files,
            pd.concat(
                (pd.read_csv(path_to_y + str(f), sep=",", header=None) for f in files)
            ).values.tolist(),
        ]
    )
)
# reading csv containing object's presence/absence in each frame
x = pd.read_csv(str(path_to_x) + "image_vector.csv", sep=",", header=None)
y = remove_first_col(y_full)

In [6]:
x.head()
y.head()

,1,2,3
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,0


In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=y,
)

In [8]:
pd.concat([y_train, x_train],axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [9]:
pd.concat([y_test, x_test],axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

In [10]:
import xgboost as xgb
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()
y_train = y_train.to_numpy()
number_of_trees = 1000
best_param = {
    'alpha': 0,
    'n_estimators': 1000,
    'booster': 'gbtree',
    'silent': 1,
    'nthread': -1,
    "colsample_bytree": 0.9,
    "gamma": 0.7000000000000001,
    "learning_rate": 0.6000000000000001,
    "max_depth": 4,
    "reg_lambda": 2,
    "subsample": 0.5,
    'objective': 'reg:pseudohubererror',
    'eval_metric': 'mphe'}
print(np.shape(x_train), np.shape(y_train), np.shape(x_test))
bt = xgb.XGBClassifier( 
    alpha = best_param['alpha'],
    n_estimators = best_param['n_estimators'],
    booster =best_param['booster'],
    verbosity = 0,
    n_jobs = best_param['nthread'],
    colsample_bytree = best_param["colsample_bytree"],
    gamma = best_param["gamma"],
    learning_rate = best_param["learning_rate"],
    max_depth = best_param["max_depth"],
    reg_lambda= best_param["reg_lambda"],
    subsample=best_param["subsample"],
    objective = 'reg:pseudohubererror',
    eval_metric = 'mphe')   # Setup xgboost model

bt.fit(x_train, y_train, # Train it to our data
       eval_set=[(x_test, y_test)], 
       verbose=True)

(800, 80) (800, 3) (200, 80)
[0]	validation_0-mphe:0.07433
[1]	validation_0-mphe:0.06616
[2]	validation_0-mphe:0.06604
[3]	validation_0-mphe:0.06641
[4]	validation_0-mphe:0.06566
[5]	validation_0-mphe:0.06523
[6]	validation_0-mphe:0.06574
[7]	validation_0-mphe:0.06626
[8]	validation_0-mphe:0.06645
[9]	validation_0-mphe:0.06592
[10]	validation_0-mphe:0.06582
[11]	validation_0-mphe:0.06574
[12]	validation_0-mphe:0.06556
[13]	validation_0-mphe:0.06515
[14]	validation_0-mphe:0.06524
[15]	validation_0-mphe:0.06482
[16]	validation_0-mphe:0.06592
[17]	validation_0-mphe:0.06589
[18]	validation_0-mphe:0.06659
[19]	validation_0-mphe:0.06658
[20]	validation_0-mphe:0.06580
[21]	validation_0-mphe:0.06568
[22]	validation_0-mphe:0.06535
[23]	validation_0-mphe:0.06508
[24]	validation_0-mphe:0.06622
[25]	validation_0-mphe:0.06624
[26]	validation_0-mphe:0.06677
[27]	validation_0-mphe:0.06682
[28]	validation_0-mphe:0.06681
[29]	validation_0-mphe:0.06682
[30]	validation_0-mphe:0.06682
[31]	validation_0-mp

XGBClassifier(alpha=0, base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mphe', feature_types=None,
              gamma=0.7000000000000001, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.6000000000000001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
              objective='reg:pseudohubererror', ...)

In [11]:
import joblib
model_file_name = "DEMO-local-xgboost"
joblib.dump(bt, model_file_name)

['DEMO-local-xgboost']

In [12]:
## Download model zipped
import botocore
try:
    s3.Bucket(bucket).download_file(key, 'model.tar.gz')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")

NameError: name 'key' is not defined

In [17]:
## Unzip model file
!tar -xvf model.tar.gz

DEMO-local-xgboost


In [12]:
import numpy as np

point_x = x_test[0:1]

print(point_x)

#point_X = np.expand_dims(point_X, axis=0)
#print(point_X)
#point_y = test_y[0]

np.savetxt("test_point.csv", point_x, delimiter=",")

[[4. 0. 3. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [13]:
#import json
import numpy as np


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    mypayload = [np.loadtxt(f, delimiter=",")]
    
print(mypayload)    

[array([4., 0., 3., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])]


In [14]:
import joblib
import xgboost

mymodel = joblib.load(model_file_name)
mymodel.predict(mypayload)

array([[0., 1., 1.]])

In [16]:
mymodel._Booster.save_model(model_file_name)
!tar czvf model.tar.gz $model_file_name

DEMO-local-xgboost


In [15]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [16]:
from sagemaker import image_uris
container = image_uris.retrieve("xgboost",boto3.Session().region_name, "0.90-2")

In [17]:
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}
print(role)
create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

#print(create_model_response2["ModelArn"])


https://s3-eu-central-1.amazonaws.com/behaviour-project/DEMO-xgboost/DEMO-local-xgboost/model.tar.gz
arn:aws:iam::444133344330:role/service-role/SageMaker-MLops


In [18]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoostEndpointConfig-2023-07-15-23-45-58
Endpoint Config Arn: arn:aws:sagemaker:eu-central-1:444133344330:endpoint-config/demo-xgboostendpointconfig-2023-07-15-23-45-58


In [19]:
import time

endpoint_name = "DEMO-XGBoostEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoostEndpoint-2023-07-15-23-51-57
arn:aws:sagemaker:eu-central-1:444133344330:endpoint/demo-xgboostendpoint-2023-07-15-23-51-57
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:eu-central-1:444133344330:endpoint/demo-xgboostendpoint-2023-07-15-23-51-57
Status: InService


In [20]:
runtime_client = boto3.client("sagemaker-runtime")

In [27]:
%%time
import json


# customize to your test file, will be 'mnist.single.test' if use data above

file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

#with open(file_name, "r") as f:
#    mypayload = [np.loadtxt(f, delimiter=",")]
payload = np.genfromtxt("test_point.csv", delimiter=',')
body = ','.join([str(item) for item in payload])
print(len(payload))
print(body)
#payload = json.dumps(payload.tolist())
print("Payload before :\n")

print(payload)

print('here')
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=body
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("ascii")

# Unpack response
print("\nPredicted Class Probabilities: {}.".format(result))

80
4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Payload before :

[4. 0. 3. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
here


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary and could not load the entire response body. See https://eu-central-1.console.aws.amazon.com/cloudwatch/home?region=eu-central-1#logEventViewer:group=/aws/sagemaker/Endpoints/DEMO-XGBoostEndpoint-2023-07-15-23-51-57 in account 444133344330 for more information.

In [15]:
from sagemaker.xgboost.model import XGBoostModel
from time import gmtime, strftime
endpoint_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

xgb_inference_model = XGBoostModel(
    model_data= "s3://behaviour-project/DEMO-xgboost/DEMO-local-xgboost/model.tar.gz",
    role=role,
    entry_point="inference.py",
    framework_version="1.7-1",
)

In [16]:
predictor = xgb_inference_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

--------------------------------------------*

UnexpectedStatusException: Error hosting endpoint DEMO-local-xgboost2023-07-12-22-51-00: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [ ]:
from sagemaker.predictor import csv_serializer
predictor.content_type = 'text/csv' # set the data type for an inference
predictor.serializer = csv_serializer # set the serializer type
predictions = predictor.predict(x_test).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

In [7]:
container = sagemaker.image_uris.retrieve("xgboost", region, "0.90-2")

from time import gmtime, strftime
model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3.{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

NameError: name 'model_file_name' is not defined

In [49]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoostEndpointConfig-2023-07-04-12-39-55
Endpoint Config Arn: arn:aws:sagemaker:eu-central-1:444133344330:endpoint-config/demo-xgboostendpointconfig-2023-07-04-12-39-55


In [50]:
%%time
import time

endpoint_name = "DEMO-XGBoostEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f'endpoint description:{resp}')
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoostEndpoint-2023-07-04-12-39-57
arn:aws:sagemaker:eu-central-1:444133344330:endpoint/demo-xgboostendpoint-2023-07-04-12-39-57
endpoint description:{'EndpointName': 'DEMO-XGBoostEndpoint-2023-07-04-12-39-57', 'EndpointArn': 'arn:aws:sagemaker:eu-central-1:444133344330:endpoint/demo-xgboostendpoint-2023-07-04-12-39-57', 'EndpointConfigName': 'DEMO-XGBoostEndpointConfig-2023-07-04-12-39-55', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2023, 7, 4, 14, 39, 57, 550000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2023, 7, 4, 14, 39, 57, 550000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'e73f8055-cc60-4cf1-bcc0-62392b40bf0c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e73f8055-cc60-4cf1-bcc0-62392b40bf0c', 'content-type': 'application/x-amz-json-1.1', 'content-length': '333', 'date': 'Tue, 04 Jul 2023 12:39:57 GMT'}, 'RetryAttempts': 0}}
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagema

In [5]:
runtime_client = boto3.client("runtime.sagemaker")

In [53]:
"""
import json
point_x = x_test.iloc[0].to_frame()
print(f'Before transposing:{point_x.shape}')
point_x = point_x.transpose()
print(f'After transposing:{point_x.shape}')
json_data = point_x.to_json(orient='records')
payload = json.dumps(json_data).encode('utf-8')
"""
import numpy as np

point_X = x_test[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = y_test[0]
np.savetxt("test_point.csv", point_X, delimiter=",")
print(point_X)

[[4. 0. 3. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [29]:
import json


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()

print(payload)
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType='text/csv', Body=payload
)
result = response["Body"].read().decode("ascii")
print("Predicted Class Probabilities: {}.".format(result))

4.000000000000000000e+00,0.000000000000000000e+00,3.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary and could not load the entire response body. See https://eu-central-1.console.aws.amazon.com/cloudwatch/home?region=eu-central-1#logEventViewer:group=/aws/sagemaker/Endpoints/DEMO-XGBoostEndpoint-2023-07-13-21-52-16 in account 444133344330 for more information.

In [3]:
sm_client = boto3.client("sagemaker")
response = sm_client.list_endpoints(StatusEquals='InService')

# Extract the endpoint names from the response
endpoint_names = [endpoint['EndpointName'] for endpoint in response['Endpoints']]
print(endpoint_names)

for e in endpoint_names:
    sm_client.delete_endpoint(EndpointName=e)

['DEMO-XGBoostEndpoint-2023-07-15-23-51-57']
